# Easy Tutorial on Distributed Training in JAX with PyTorch Frame Data Loading

Author: Yiwen Yuan
Suggested Device: TPU VM v3-8

[PyTorch Frame](https://github.com/pyg-team/pytorch-frame) is a deep learning extension for PyTorch, designed for heterogeneous tabular data with different column type. It is super easy to load tables into PyTorch Frame and perform necessary stats computation for different semantic types. 

This tutorial is a simple example of distributed training in JAX using PyTorch Frame as data loader.

In [1]:
%%capture
!pip install pytorch_frame

In [2]:
# Import necessary libraries

import time
from functools import partial

import jax
import jax.numpy as jnp
import pandas as pd
import torch
from jax import jit, random, value_and_grad, vmap
from jax.nn import swish
from tqdm import tqdm

from torch_frame import categorical, numerical, timestamp
from torch_frame.data import DataLoader, Dataset
from torch_frame.data.stats import StatType

In [3]:
# Constants <3

NUM_FEAT = 24
NUM_DEVICES = jax.device_count()
BATCH_SIZE = 512
# Two layer simple MLP
LAYER_SIZES = [24, 1024, 1024, 1]
PARAM_SCALE = 0.01
INIT_LR = 0.0001
DECAY_RATE = 0.95
DECAY_STEPS = 5
NUM_EPOCHS = 5
TARGET_COL = 'Premium Amount'

E0000 00:00:1733984901.725559      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
E1212 06:28:21.760614440      77 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2024-12-12T06:28:21.760595515+00:00"}


In [4]:
print(f"Number of devices: {NUM_DEVICES}")

Number of devices: 8


**Data Loading with PyTorch Frame**

To load data with PyTorch Frame, you need to specify the semantic column types.

In [5]:
col_to_stype = {
    'Age': numerical,
    'Annual Income': numerical,
    'Marital Status': categorical,
    'Number of Dependents': numerical,
    'Education Level': categorical,
    'Occupation': categorical,
    'Health Score': numerical,
    'Location': categorical,
    'Policy Type': categorical,
    'Previous Claims': numerical,
    'Vehicle Age': numerical,
    'Credit Score': numerical,
    'Insurance Duration': numerical,
    'Policy Start Date': timestamp,
    'Customer Feedback': categorical,
    'Smoking Status': categorical,
    'Exercise Frequency': categorical,
    'Property Type': categorical
}
test_dataset = Dataset(df=pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv'),
                       col_to_stype=col_to_stype)
col_to_stype = col_to_stype.copy()
col_to_stype['Premium Amount'] = numerical

dataset = Dataset(df=pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv'),
                  col_to_stype=col_to_stype, target_col=TARGET_COL)

# Saves the materialized tensors for easy reuse
dataset.materialize(path='/kaggle/working/data.pt')
test_dataset.materialize(path='/kaggle/working/test_data.pt')

Dataset()

**Nan Imputation**

The original data contains nans. We need to fill the nan values. We use MEAN to impute nan for numerical columns, MODE for categorical columns and MEDIAN for timestamp columns. The stats are already calculated as part of the PyTorch Frame data materialization process.

In [6]:
# Use mean value to fill nans in numerical columns
# Use mode value to fill nans in categorical columns
# Use newest time to fill nans in timestamp columns
fill_vals = []
means = []
stds = []
for stype in [numerical, categorical, timestamp]:
    col_names = dataset.tensor_frame.col_names_dict[stype]
    for col_name in col_names:
        if stype == numerical:
            fill_vals.append(dataset.col_stats[col_name][StatType.MEAN])
            means.append(dataset.col_stats[col_name][StatType.MEAN])
            stds.append(dataset.col_stats[col_name][StatType.STD])
        elif stype == categorical:
            fill_vals.append(0)
            means.append(0.)
            stds.append(1.)
        elif stype == timestamp:
            fill_vals += dataset.col_stats[col_name][StatType.NEWEST_TIME]
            means += dataset.col_stats[col_name][StatType.MEDIAN_TIME]
            stds += [1.] * 7
        else:
            raise ValueError("Unsupported stype")
fill_vals = jnp.array(fill_vals)
means = jnp.array(means)
stds = jnp.array(stds)

y_mean = torch.mean(dataset.tensor_frame.y).cpu().numpy()
y_std = torch.std(dataset.tensor_frame.y).cpu().numpy()

**2-layer MLP in JAX with distributed training capabilities**

In [7]:
def init_network_params(sizes, key=random.PRNGKey(0), scale=1e-2):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return (scale * random.normal(w_key, (n, m)),
                scale * random.normal(b_key, (n, )))

    keys = random.split(key, len(sizes))
    return [
        random_layer_params(m, n, k, scale)
        for m, n, k in zip(sizes[:-1], sizes[1:], keys)
    ]

def predict(params, image):
    activations = image
    for w, b in params[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = swish(outputs)

    final_w, final_b = params[-1]
    logits = jnp.dot(final_w, activations) + final_b
    return logits

batched_predict = vmap(predict, in_axes=(None, 0))

def loss(params, images, targets):
    logits = batched_predict(params, images)
    return jnp.mean(jnp.abs(logits - targets))

@partial(jax.pmap, axis_name='devices', in_axes=(None, 0, 0, None),
         out_axes=(None, 0))
def update(params, x, y, epoch_number):
    loss_value, grads = value_and_grad(loss)(params, x, y)
    grads = [(jax.lax.psum(dw, 'devices') / NUM_DEVICES,
              jax.lax.psum(db, 'devices') / NUM_DEVICES)
             for dw, db in grads]
    lr = INIT_LR * DECAY_RATE**(epoch_number / DECAY_STEPS)
    return [(w - lr * dw, b - lr * db)
            for (w, b), (dw, db) in zip(params, grads)], loss_value

@jit
def batched_mae(params, images, targets):
    images = jnp.reshape(images, (len(images), NUM_FEAT))
    predicted_targets = batched_predict(params, images)
    return jnp.mean(jnp.abs(predicted_targets - targets))


def mae(params, data_loader):
    maes = []
    for tf in data_loader:
        x = torch.cat([
            tf.feat_dict[categorical], tf.feat_dict[numerical],
            tf.feat_dict[timestamp].squeeze(1)
        ], dim=1).numpy()
        y = (tf.y.numpy() - y_mean) / y_std
        nan_mask = jnp.isnan(x)
        if x.any():
            x = jnp.where(nan_mask, fill_vals, x)
        x = (x - means) / stds
        maes.append(batched_mae(params, x, y))
    return jnp.mean(jnp.array(maes))

In [8]:
init_params = init_network_params(LAYER_SIZES, random.PRNGKey(0),
                                  scale=PARAM_SCALE)

**Actual Training**

In [9]:
train_loader = DataLoader(dataset.tensor_frame, batch_size=BATCH_SIZE,
                          shuffle=True, drop_last=True)
params = init_params
for epoch in range(1, NUM_EPOCHS + 1):
    start_time = time.time()
    losses = []
    for tf in tqdm(train_loader):
        x = torch.cat([
            tf.feat_dict[categorical], tf.feat_dict[numerical],
            tf.feat_dict[timestamp].squeeze(1)
        ], dim=1).numpy()
        y = (tf.y.numpy() - y_mean) / y_std
        nan_mask = jnp.isnan(x)
        if nan_mask.any():
            x = jnp.where(nan_mask, fill_vals, x)
        x = (x - means) / stds
        x = jnp.reshape(x, (NUM_DEVICES, BATCH_SIZE // NUM_DEVICES, NUM_FEAT))
        y = jnp.reshape(y, (NUM_DEVICES, BATCH_SIZE // NUM_DEVICES, 1))
        params, loss_value = update(params, x, y, epoch)
        losses.append(jnp.sum(loss_value))
    epoch_time = time.time() - start_time
    train_mae = mae(params, train_loader)
    print(f"Epoch {epoch} in {epoch_time:0.2f} sec")
    print(f"Training set loss {jnp.mean(jnp.array(losses))}")
    print(f"Training set mae {train_mae}")


  0%|          | 0/2343 [00:00<?, ?it/s]

  0%|          | 1/2343 [00:00<28:35,  1.37it/s]

  1%|          | 18/2343 [00:00<01:20, 28.81it/s]

  1%|▏         | 35/2343 [00:00<00:41, 55.29it/s]

  2%|▏         | 52/2343 [00:01<00:28, 79.52it/s]

...

 99%|█████████▉| 2319/2343 [00:13<00:00, 168.36it/s]

100%|█████████▉| 2336/2343 [00:14<00:00, 160.65it/s]

100%|██████████| 2343/2343 [00:14<00:00, 164.62it/s]

Epoch 5 in 14.24 sec


Training set loss 5.901015281677246
Training set mae 0.7477869391441345


**Generating Predictions on Test Data**

In [10]:
test_loader = DataLoader(test_dataset.tensor_frame, batch_size=BATCH_SIZE,
                         shuffle=False, drop_last=False)
results = []

parallel_predict = jax.pmap(vmap(predict, in_axes=(None, 0)), in_axes=(None, 0))

for tf in tqdm(test_loader):
    x = torch.cat([
        tf.feat_dict[categorical], tf.feat_dict[numerical],
        tf.feat_dict[timestamp].squeeze(1)
    ], dim=1).numpy()
    nan_mask = jnp.isnan(x)
    if nan_mask.any():
        x = jnp.where(nan_mask, fill_vals, x)
    x = (x - means) / stds
    x = jnp.reshape(x, (NUM_DEVICES, len(x) // NUM_DEVICES, NUM_FEAT))
    result = parallel_predict(params, x).reshape(-1) * y_std + y_mean
    results.append(result)

outputs = jnp.concatenate(results)
submission = pd.read_csv("/kaggle/input/playground-series-s4e12/sample_submission.csv")
submission[TARGET_COL] = outputs
submission = submission[['id', TARGET_COL]]
submission.to_csv('/kaggle/working/final_submission.csv', index=False)

  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 1/1563 [00:00<07:08,  3.64it/s]

  1%|▏         | 21/1563 [00:00<00:22, 70.04it/s]

  3%|▎         | 41/1563 [00:00<00:13, 111.97it/s]

  4%|▍         | 61/1563 [00:00<00:10, 139.29it/s]

...

 95%|█████████▍| 1484/1563 [00:07<00:00, 196.40it/s]

 96%|█████████▌| 1504/1563 [00:07<00:00, 195.74it/s]

 98%|█████████▊| 1524/1563 [00:08<00:00, 195.52it/s]

 99%|█████████▉| 1544/1563 [00:08<00:00, 195.80it/s]

100%|██████████| 1563/1563 [00:08<00:00, 180.67it/s]